In [18]:
import pandas as pd

##### 0. base 작업
Respiratory 진단 받은 환자의 SUBJECT_ID, HADM_ID, TLOS, AGE, DOA(Dead or Alive) column 생성 및 추출(총 7,631명)

In [19]:
# respiratory 환자 사망률
print("호흡기 질환 환자의 사망률")
print("-"*59)
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["respiratory", "Respiratory"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Respiratory 관련 ICD9 code 수: ", len(disease_list))
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])
df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Respiratory 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["TLOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["TLOS"] >= "1days"
df6 = df5[over].sort_values(by=["TLOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("TLOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"

df10 = df9[["SUBJECT_ID", "HADM_ID", "TLOS", "AGE", "DOA"]]
print("-"*59)
print("호흡기 질환 환자의 가공된 Admission table dataframe")
df10
# df10[["SUBJECT_ID", "HADM_ID"]].to_csv("~/project/MIMIC-III/Data/SUBJECTID.csv", index=False)

호흡기 질환 환자의 사망률
-----------------------------------------------------------
Respiratory 관련 ICD9 code 수:  92
Respiratory 환자 수(최초 입원):  10329
TLOS가 24H 이상인 환자 수:  9932
18세 이상 89세 미만 환자 수:  7631
-----------------------------------------------------------
호흡기 질환 환자의 가공된 Admission table dataframe


,SUBJECT_ID,HADM_ID,TLOS,AGE,DOA
6392,20936,157548,16 days 06:15:00,18,NaT
582,25600,122994,2 days 12:49:00,18,NaT
1012,79166,177249,3 days 11:02:00,18,NaT
3214,75775,101668,7 days 13:07:00,18,NaT
5057,26861,115381,11 days 16:01:00,18,NaT
...,...,...,...,...,...
6091,48806,116558,14 days 23:36:00,88,NaT
3008,19067,192309,7 days 01:22:00,88,NaT
2138,40370,199999,5 days 12:33:00,88,NaT
6524,47335,161669,16 days 20:28:00,88,16 days 20:28:00


##### 1. 인구통계학 데이터 추출 1
-> SUBJECT_ID, HADM_ID, AGE, GENDER, ETHNICITY, TLOS(병원 전체), LOS(ICU 전체), DOA(사망일까지 걸린 시간), HOSPITAL_EXPIRE_FLAG(병원 내 사망 기록) column 추출(총 7,631명)
1. ADMISSIONS(가공)
2. ADMISSIONS(원본)
3. PATIENTS
4. DIAGNOSES_ICD

In [20]:
df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df2 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df3= pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ICUSTAYS.csv")

In [21]:
# Respiratory 진단 받은 환자의 AMISSIONS, PATIENTS, ICUSTAYS 합치기
# ICUSTAYS merge 과정에서
print("ADMISSIONS(가공) table 환자 수: ", len(df10))
df6 = pd.merge(df10, df1[["SUBJECT_ID", "HADM_ID", "ETHNICITY", "HOSPITAL_EXPIRE_FLAG"]], on=['SUBJECT_ID', 'HADM_ID']).sort_values(by='SUBJECT_ID')
print("ADMISSIONS(원본) table 추가: ", len(df6))
df7 = pd.merge(df6, df2[["SUBJECT_ID", "GENDER"]], on=['SUBJECT_ID'])
print("PATIENTS table 추가: ", len(df7))

df3 = df3[["SUBJECT_ID", "HADM_ID", "LOS"]]
df8 = df3.groupby(["SUBJECT_ID","HADM_ID"])['LOS'].sum().reset_index()
df9 = pd.merge(df7, df8, on=['SUBJECT_ID', 'HADM_ID'], how='left')
df = df9.fillna({'LOS': 0}) # 결측치 있는 값은 0으로 채우기
# df9.sort_values(by='LOS') # 결측치가 잘 채워졌는지 확인하는 코드
print("ICUSTAYS table 추가: ", len(df))

print("-"*117)
print("호흡기 질환 환자의 인구통계학 데이터 table")
df

ADMISSIONS(가공) table 환자 수:  7631
ADMISSIONS(원본) table 추가:  7631
PATIENTS table 추가:  7631
ICUSTAYS table 추가:  7631
---------------------------------------------------------------------------------------------------------------------
호흡기 질환 환자의 인구통계학 데이터 table


,SUBJECT_ID,HADM_ID,TLOS,AGE,DOA,ETHNICITY,HOSPITAL_EXPIRE_FLAG,GENDER,LOS
0,36,165660,10 days 01:46:00,72,NaT,WHITE,0,M,4.3483
1,91,121205,17 days 15:08:00,81,17 days 15:08:00,WHITE,1,F,3.9715
2,94,140037,22 days 22:53:00,75,NaT,ASIAN,0,M,22.8977
3,101,175533,15 days 18:41:00,82,15 days 18:41:00,ASIAN,1,M,9.8919
4,111,192123,11 days 04:50:00,66,NaT,WHITE,0,F,10.5708
...,...,...,...,...,...,...,...,...,...
7626,99897,181057,5 days 00:02:00,54,NaT,BLACK/HAITIAN,0,M,0.8342
7627,99899,188409,9 days 01:34:00,87,9 days 01:34:00,BLACK/AFRICAN AMERICAN,1,M,9.1439
7628,99912,189380,13 days 21:35:00,84,NaT,WHITE,0,M,12.0867
7629,99939,159023,11 days 10:59:00,26,NaT,HISPANIC OR LATINO,0,M,2.8780


##### 2. 인구통계학 데이터 추출 2
-> CHARTEVENTS table에서 키, 체중 및 체질량 지수(BMI)column 추가(총 ) <br>
-> 리눅스 awk 활용해 CHARTEVENTS table에서 키, 체중 데이터 추출 <br>
-> 추출한 키, 체중 데이터 바탕으로 BMI 계산

In [22]:
df1 = pd.read_csv("~/project/MIMIC-III/Data/height_weight.csv")
# 결측치 제거
df1 = df1.dropna(subset=['weight'])
df1 = df1.dropna(subset=['height'])

# df와 결합
df2 = pd.merge(df, df1, on=["SUBJECT_ID", "HADM_ID"])
df2

# BMI column 생성
df2["BMI"] = df2['weight'] / (df2['height']/100) ** 2

# 칼럼명 대문자로 통일
df2 = df2.rename(columns={'weight': 'WEIGHT', 'height':'HEIGHT'})
df2

# 칼럼 순서 변경
df2 = df2.reindex(columns=['SUBJECT_ID', 'HADM_ID', 'ETHNICITY', 'GENDER', 'AGE', 'HEIGHT', 'WEIGHT', 'BMI', 'LOS', 'TLOS', 'HOSPITAL_EXPIRE_FLAG', 'DOA'])
df2

,SUBJECT_ID,HADM_ID,ETHNICITY,GENDER,AGE,HEIGHT,WEIGHT,BMI,LOS,TLOS,HOSPITAL_EXPIRE_FLAG,DOA
0,36,165660,WHITE,M,72,180.0,105.300000,32.500000,4.3483,10 days 01:46:00,0,NaT
1,188,123860,WHITE,M,55,173.0,80.400000,26.863577,3.1676,8 days 07:16:00,0,NaT
2,502,116367,WHITE,M,50,178.0,85.850000,27.095695,11.9479,11 days 20:23:00,1,11 days 20:23:00
3,890,195962,WHITE,M,67,168.0,96.500000,34.190760,9.7871,15 days 18:31:00,0,NaT
4,975,175734,WHITE,M,68,180.0,118.180000,36.475309,27.4275,36 days 22:22:00,1,36 days 22:22:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1772,99863,100749,WHITE - BRAZILIAN,M,42,183.0,122.829167,36.677466,28.7306,35 days 15:43:00,0,NaT
1773,99868,177777,UNKNOWN/NOT SPECIFIED,F,20,140.0,51.700000,26.377551,1.3862,3 days 10:59:00,0,NaT
1774,99873,143544,WHITE,M,45,188.0,110.333333,31.216991,2.0048,5 days 00:47:00,0,NaT
1775,99881,172327,WHITE,M,77,170.0,109.500000,37.889273,2.1914,2 days 03:27:00,1,2 days 03:27:00


##### 3. Demographic 데이터 저장

In [23]:
df = df2

##### 4. 호흡기 질환 환자의 사망률 확인

In [24]:
# 특정 기간내 사망률 확인
print("호흡기 질환 환자의 사망률")
print("호흡기 질환 환자 수: ", len(df))
print("인구통계학적 데이터 column: ", len(df.columns))
print("인구통계학적 데이터 column name: ", df.columns.values)
print("-"*80)

# 30일 이내 사망
less = df["DOA"] <= "30days"
df11 = df[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-"*80)

# 60일 이내 사망
less = df["DOA"] <= "60days"
df12 = df[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-"*80)

호흡기 질환 환자의 사망률
호흡기 질환 환자 수:  1777
인구통계학적 데이터 column:  12
인구통계학적 데이터 column name:  ['SUBJECT_ID' 'HADM_ID' 'ETHNICITY' 'GENDER' 'AGE' 'HEIGHT' 'WEIGHT' 'BMI'
 'LOS' 'TLOS' 'HOSPITAL_EXPIRE_FLAG' 'DOA']
--------------------------------------------------------------------------------
30일 이내 사망자 수:  369
30일 이내 사망자 수 비율:  4.8355392478050065
--------------------------------------------------------------------------------
60일 이내 사망자 수:  422
60일 이내 사망자 수 비율:  5.530074695321714
--------------------------------------------------------------------------------
